<a href="https://colab.research.google.com/github/Rohit-Saswadkar/Data-Manipulation-using-Pyspark/blob/main/Banking_data_Transforming_and_Analysis_using_Pyspark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Let's get start**

In [192]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Install pyspark**


In [193]:
!pip install pyspark
!pip install openpyxl

In [194]:
# set the spark session
from pyspark.sql import SparkSession

sp = SparkSession.builder \
    .appName("Banking_Analysis_using_Pyspark") \
    .getOrCreate()

In [195]:
# Configure the SparkSession
sp.conf.set("spark.sql.shuffle.partitions", "50")

In [196]:
# Configure the logging settings
import logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


##**Loading Dataset**

In [197]:
# load df paths
path1 = '/content/drive/MyDrive/Py Spark/Banking Project/BankingDataset_1.csv'
path2 = '/content/drive/MyDrive/Py Spark/Banking Project/BankingDataset_2.csv'

In [198]:
df1 = sp.read.csv(path1, header=True, inferSchema=True, sep=",", ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True)
df2 = sp.read.csv(path2, header=True, inferSchema=True, sep=",", ignoreLeadingWhiteSpace=True, ignoreTrailingWhiteSpace=True)

In [199]:
# check schema of the df's
df1.printSchema()
df2.printSchema()

root
 |-- TransactionID: integer (nullable = true)
 |-- AccountNumber: integer (nullable = true)
 |-- TransactionType: string (nullable = true)
 |-- Amount: double (nullable = true)
 |-- TransactionDate: timestamp (nullable = true)
 |-- BranchCode: integer (nullable = true)
 |-- Currency: string (nullable = true)
 |-- TransactionTime: integer (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)

root
 |-- AccountNumber: integer (nullable = true)
 |-- AccountHolder: string (nullable = true)
 |-- AccountType: string (nullable = true)
 |-- Balance: double (nullable = true)
 |-- InterestRate: double (nullable = true)
 |-- CreditScore: integer (nullable = true)
 |-- OpeningDate: string (nullable = true)
 |-- LoanAmount: double (nullable = true)
 |-- AccountHolderDetails: string (nullable = true)



In [200]:
# drop unnecessary columns
columns_to_drop = ['_c9', '_c10', '_c11', '_c12', '_c13', '_c14']
df2 = df2.drop(*columns_to_drop)

In [201]:
columns_to_drop = ['_C8','_C9']

df1 = df1.drop(* columns_to_drop)

##**Initial data exploration**

In [202]:
df1.describe().show() # check the data
df2.describe().show()

+-------+-----------------+------------------+---------------+------------------+------------------+--------+-----------------+
|summary|    TransactionID|     AccountNumber|TransactionType|            Amount|        BranchCode|Currency|  TransactionTime|
+-------+-----------------+------------------+---------------+------------------+------------------+--------+-----------------+
|  count|             1000|              1000|            940|               940|              1000|    1000|             1000|
|   mean|            500.5|        560160.735|           NULL| 5063.896222674368|           253.436|  -999.0|           11.863|
| stddev|288.8194360957494|260670.28320861366|           NULL|2847.0992164759687|144.11454092111913|     0.0|6.515726029053529|
|    min|                1|            100268|        Deposit|        138.777986|                 1|    *^$$|                1|
|    max|             1000|            996071|     Withdrawal|        9987.02423|               499|    

In [203]:
df1.dtypes # data types of columns

[('TransactionID', 'int'),
 ('AccountNumber', 'int'),
 ('TransactionType', 'string'),
 ('Amount', 'double'),
 ('TransactionDate', 'timestamp'),
 ('BranchCode', 'int'),
 ('Currency', 'string'),
 ('TransactionTime', 'int')]

In [204]:
df2.dtypes

[('AccountNumber', 'int'),
 ('AccountHolder', 'string'),
 ('AccountType', 'string'),
 ('Balance', 'double'),
 ('InterestRate', 'double'),
 ('CreditScore', 'int'),
 ('OpeningDate', 'string'),
 ('LoanAmount', 'double'),
 ('AccountHolderDetails', 'string')]

In [205]:
df1 = df1.dropDuplicates() # drop duplicates to enhance performance
df2 = df2.dropDuplicates()

In [206]:
# check unique counts of each column of DF1
unique_counts = {}
for col_name in df1.columns:
    unique_count = df1.select(col_name).distinct().count()
    unique_counts[col_name] = unique_count

# Display the unique counts
for col_name, unique_count in unique_counts.items():
    print(f"Column {col_name} has {unique_count} unique values")


Column TransactionID has 1000 unique values
Column AccountNumber has 558 unique values
Column TransactionType has 5 unique values
Column Amount has 941 unique values
Column TransactionDate has 1000 unique values
Column BranchCode has 431 unique values
Column Currency has 7 unique values
Column TransactionTime has 23 unique values


In [207]:
# check unique counts of each column of DF2
unique_counts = {}
for col_name in df2.columns:
    unique_count = df2.select(col_name).distinct().count()
    unique_counts[col_name] = unique_count

# Display the unique counts
for col_name, unique_count in unique_counts.items():
    print(f"Column {col_name} has {unique_count} unique values")


Column AccountNumber has 689 unique values
Column AccountHolder has 336 unique values
Column AccountType has 4 unique values
Column Balance has 689 unique values
Column InterestRate has 689 unique values
Column CreditScore has 389 unique values
Column OpeningDate has 689 unique values
Column LoanAmount has 689 unique values
Column AccountHolderDetails has 377 unique values


##**Data Cleaning and Preprocessing**

###**Check ulll values in columns**

In [208]:
from pyspark.sql.functions import col, when, count, isnull

# Show columns with missing values for df1
df1.select([count(when(isnull(c), c)).alias(c) for c in df1.columns]).show()

# Show columns with missing values for df2
df2.select([count(when(isnull(c), c)).alias(c) for c in df2.columns]).show()


+-------------+-------------+---------------+------+---------------+----------+--------+---------------+
|TransactionID|AccountNumber|TransactionType|Amount|TransactionDate|BranchCode|Currency|TransactionTime|
+-------------+-------------+---------------+------+---------------+----------+--------+---------------+
|            0|            0|             60|    60|              0|         0|       0|              0|
+-------------+-------------+---------------+------+---------------+----------+--------+---------------+

+-------------+-------------+-----------+-------+------------+-----------+-----------+----------+--------------------+
|AccountNumber|AccountHolder|AccountType|Balance|InterestRate|CreditScore|OpeningDate|LoanAmount|AccountHolderDetails|
+-------------+-------------+-----------+-------+------------+-----------+-----------+----------+--------------------+
|            0|            0|          0|      0|           0|          0|          0|         0|                   0

###**A] Null value imputation**

**1) Identifying Numeric and Categorical Columns in DF! and replace null with them**

In [209]:
# separating the categorical and numerical columns by their datatype
from pyspark.sql.types import IntegerType, DoubleType, StringType

numeric_cols = [field.name for field in df1.schema.fields if isinstance(field.dataType, (IntegerType, DoubleType))]
categorical_cols = [field.name for field in df1.schema.fields if isinstance(field.dataType, StringType)]


In [210]:
# calculating median for numerical columns
numeric_medians = {}
for col in numeric_cols:
    median = df1.approxQuantile(col, [0.5], 0.01)[0]
    numeric_medians[col] = median


In [211]:
# calculating mode for categorical columns
from pyspark.sql.functions import col, count, desc

categorical_modes = {}
for col_name in categorical_cols:
    mode_df = df1.groupBy(col_name).count().orderBy(desc("count")).first()
    if mode_df:
        categorical_modes[col_name] = mode_df[col_name]


In [212]:
# replaced null with median and mode repectively
df1_filled = df1.fillna(numeric_medians)
df1_filled = df1_filled.fillna(categorical_modes)


In [213]:
# Show columns with missing values for df1
df1_filled.select([count(when(isnull(c), c)).alias(c) for c in df1.columns]).show()

+-------------+-------------+---------------+------+---------------+----------+--------+---------------+
|TransactionID|AccountNumber|TransactionType|Amount|TransactionDate|BranchCode|Currency|TransactionTime|
+-------------+-------------+---------------+------+---------------+----------+--------+---------------+
|            0|            0|              0|     0|              0|         0|       0|              0|
+-------------+-------------+---------------+------+---------------+----------+--------+---------------+



**B] Creating new Df by merging df1 and df2 using AccountNumber**

In [214]:
df = df1_filled.join( df2, on='AccountNumber', how='inner')

In [215]:
df.show()

+-------------+-------------+---------------+-----------+-------------------+----------+--------+---------------+-----------------+-----------+-----------+------------+-----------+---------------+-----------+--------------------+
|AccountNumber|TransactionID|TransactionType|     Amount|    TransactionDate|BranchCode|Currency|TransactionTime|    AccountHolder|AccountType|    Balance|InterestRate|CreditScore|    OpeningDate| LoanAmount|AccountHolderDetails|
+-------------+-------------+---------------+-----------+-------------------+----------+--------+---------------+-----------------+-----------+-----------+------------+-----------+---------------+-----------+--------------------+
|       334892|           42|     Withdrawal| 5037.65665|2023-02-11 00:00:00|       342|     USD|              9|Patricia Gonzalez|       Loan|25577.87469|  2.00892616|        847| 2/11/2020 0:00|39920.51583|Sector: Finance, ...|
|       334892|          595|       Transfer|6418.314831|2024-08-17 00:00:00|   

**C] Outliers checking**

In [216]:
from pyspark.sql.functions import mean, stddev, col

# Calculate mean and standard deviation for the 'Amount' column
stats = df.select(mean('Amount').alias('mean'), stddev('Amount').alias('stddev')).first()
mean_value = stats['mean']
stddev_value = stats['stddev']

# Define the filter condition for outliers (values outside the range of mean ± 3 * stddev)
outlier_condition = (col('Amount') < mean_value - 3 * stddev_value) | (col('Amount') > mean_value + 3 * stddev_value)

# Count the number of outliers
outliers_count = df.filter(outlier_condition).count()

print(f"Number of outliers: {outliers_count}")


Number of outliers: 0


**D] Extract Year , Month and Day from Transaction date**

In [217]:
df = df.withColumn('TransactionYear', year(col('TransactionDate')))
df = df.withColumn('TransactionMonth', month(col('TransactionDate')))
df = df.withColumn('TransactionDay', dayofmonth(col('TransactionDate')))


##**Exploratory Data analysis**

In [218]:
# get stats
df.describe().show()


+-------+------------------+------------------+---------------+------------------+------------------+--------+-----------------+----------------+-----------+------------------+------------------+------------------+-------------+------------------+--------------------+------------------+------------------+------------------+
|summary|     AccountNumber|     TransactionID|TransactionType|            Amount|        BranchCode|Currency|  TransactionTime|   AccountHolder|AccountType|           Balance|      InterestRate|       CreditScore|  OpeningDate|        LoanAmount|AccountHolderDetails|   TransactionYear|  TransactionMonth|    TransactionDay|
+-------+------------------+------------------+---------------+------------------+------------------+--------+-----------------+----------------+-----------+------------------+------------------+------------------+-------------+------------------+--------------------+------------------+------------------+------------------+
|  count|             

**1] Check the transaction type counts**

In [219]:
df.groupBy('TransactionType').count().show()

+---------------+-----+
|TransactionType|count|
+---------------+-----+
|        Deposit|  197|
|       Transfer|  207|
|        Payment|  277|
|     Withdrawal|  190|
+---------------+-----+



**2] Analysing Total Amount and Avg balance of each account**

In [220]:
from pyspark.sql.functions import year, month, dayofmonth, hour
from pyspark.sql.functions import sum as Fsum, count as Fcount, mean as Fmean, median as Fmedian, round as Fround, desc as Fdesc, asc as Fasc

df_aggregated = df.groupBy('AccountNumber').agg(
    Fround(Fsum('Amount'),2).alias('TotalAmount'),
    Fround(Fmean('Balance'),2).alias('AverageBalance')
)

df_aggregated.show()


+-------------+-----------+--------------+
|AccountNumber|TotalAmount|AverageBalance|
+-------------+-----------+--------------+
|       886201|    2046.35|       7451.75|
|       539316|    6179.21|       35013.1|
|       892138|    8112.54|      28482.39|
|       752678|    9250.44|      32460.37|
|       691855|   13903.83|      24558.86|
|       879077|   11755.54|      47361.73|
|       377515|   10091.66|      15140.76|
|       766598|   13506.12|      17391.03|
|       713728|    9142.21|      44291.83|
|       817886|   12974.58|       8317.01|
|       479868|   17662.21|       9528.76|
|       119374|   11882.19|        277.48|
|       386867|    6182.65|      27948.63|
|       380465|   10075.31|      41199.64|
|       717169|   11109.37|        499.41|
|       140388|   18455.01|       26414.3|
|       559443|    6023.69|      37005.32|
|       127491|     9247.4|      44932.62|
|       344720|    7535.91|      28931.18|
|       274020|    5037.66|      33638.41|
+----------

**Insights:**

- **High Total Amount Correlates with Higher Average Balance**: Accounts with higher total amounts generally have higher average balances, indicating a trend where larger transactions are associated with higher average account balances.
- **Notable Exceptions**: Some accounts, like `817886` and `119374`, have high total amounts but significantly lower average balances, suggesting occasional large transactions amidst generally lower average balances.

**3] Analysing Trends of Total Amount and no of transactions by year and montha**

In [221]:
df_trends = df.groupBy('TransactionYear', 'TransactionMonth').agg(
    Fround(Fsum('Amount'),2).alias('TotalAmount'),
    Fcount('TransactionID').alias('TransactionCount')
)
df_trends.show()

+---------------+----------------+-----------+----------------+
|TransactionYear|TransactionMonth|TotalAmount|TransactionCount|
+---------------+----------------+-----------+----------------+
|           2023|               4|   153976.9|              27|
|           2025|               1|  136148.89|              27|
|           2023|               5|  130079.14|              27|
|           2024|               7|  121591.29|              26|
|           2023|               6|  135687.85|              26|
|           2024|               1|  146239.18|              28|
|           2025|               9|  105565.48|              23|
|           2025|               7|  136955.96|              27|
|           2024|              10|  136453.02|              28|
|           2024|               9|  127071.94|              25|
|           2024|              12|  115769.17|              27|
|           2023|               8|  135001.78|              28|
|           2023|               9|  1162

Insights:

- **High Transaction Amounts in Early 2025**: The total transaction amounts peak notably in early 2025, particularly in April (`163028.91`) and January (`136148.89`), indicating a significant increase in transaction volume during this period.
  
- **Consistent Transaction Activity**: Despite fluctuations in total amounts, transaction counts are relatively stable across most months, with variations typically within a few transactions, suggesting consistent transaction activity across the months.

Summary

The data shows that while transaction amounts vary significantly, especially in early 2025, the number of transactions remains fairly steady, pointing to periods of high transaction activity.

**4] Analysing account holder names by avg loan amount as well as their avg credit scores and avg interest rates**

In [222]:
df_account_holders = df.groupby('AccountHolder').agg( Fround(Fmean('LoanAmount'),2).alias('Avg Loan Amount'), Fround(Fmean('CreditScore'),2).alias('Avg Credit score'), Fround(Fmean(
'InterestRate'),2).alias('Avg Interest Rate'), Fround(Fmean('TransactionTime'),2).alias('Avg transaction Time')).orderBy(Fdesc('Avg Loan Amount'))
df_account_holders.show()

+------------------+---------------+----------------+-----------------+--------------------+
|     AccountHolder|Avg Loan Amount|Avg Credit score|Avg Interest Rate|Avg transaction Time|
+------------------+---------------+----------------+-----------------+--------------------+
|       Sarah Jones|       49898.46|           788.0|             1.07|                17.5|
|      Linda Garcia|       49543.32|           497.0|             2.41|                11.0|
| Jessica Hernandez|       49507.93|           530.0|             0.69|                18.0|
|      Karen Garcia|       49497.29|           431.0|             0.87|                15.0|
|    Barbara Miller|       48432.58|           305.0|              2.7|                16.5|
|Patricia Hernandez|       48325.21|           620.0|             2.23|                14.0|
|   Jennifer Miller|       47938.53|           474.0|             1.18|                12.5|
|   Patricia Martin|       47465.31|           731.0|             2.01

Insights:

- **High Loan Amounts**: Account holders like Sarah Jones and Linda Garcia have the highest average loan amounts, indicating substantial borrowing.
- **Credit Score Trends**: Higher loan amounts often correlate with higher average credit scores, though some exceptions, like Barbara Miller, show lower credit scores.
- **Interest Rates and Transaction Times**: Interest rates vary widely, with some account holders paying significantly more, while transaction times remain fairly consistent across individuals.

##**Save the Transformed Dataframe**

In [223]:
# CSV format
df.write \
    .format("csv") \
    .option("header", "true") \
    .mode("overwrite") \
    .save("/content/drive/MyDrive/Py Spark/Banking Project/Final Dataframe/Banking_DF.csv")


In [224]:
# Parquet format
df.write \
    .format("parquet") \
    .mode("overwrite") \
    .save("/content/drive/MyDrive/Py Spark/Banking Project/Final Dataframe/Banking_DF.parquet")
